# Test passthrough fixed shape tensors

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [4]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework
from wallaroo.object import EntityNotFoundError

import pyarrow as pa
import numpy as np
import pandas as pd

import datetime

In [5]:
wl = wallaroo.Client()

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

In [6]:
pipeline_name = 'passthrough-vc-fixed-shape-tensor-test'
model_name = 'passthrough-vc-fixed-shape-tensor'
model_file_name = 'passthrough_vc_fixed_shape_tensors.zip'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [7]:
def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [8]:
input_schema = pa.schema([
    pa.field('image', pa.fixed_shape_tensor(pa.uint16(), [2052, 2456])),
])

output_schema = pa.schema([
    pa.field('image', pa.fixed_shape_tensor(pa.uint16(), [2052, 2456])),
    pa.field('virtual_stain', pa.fixed_shape_tensor(pa.uint8(), [2052, 2456, 3])),
    pa.field('mask_overlay', pa.fixed_shape_tensor(pa.uint8(), [2052, 2456, 3])),
    pa.field('mask', pa.fixed_shape_tensor(pa.uint16(), [2052, 2456])),
])

### Upload Model

The model will be uploaded with the framework set as `Framework.KERAS`.

In [9]:
model = wl.upload_model(model_name, model_file_name, framework=Framework.PYTHON, input_schema=input_schema, output_schema=output_schema)

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a container runtime..
Model is attempting loading to a container runtime...successful

Ready


In [10]:
model.config().runtime()

'flight'

### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [11]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(1.).memory('1Gi') \
    .sidekick_memory(model, '1Gi') \
    .sidekick_cpus(model, 1.0) \
    .build()

In [12]:
# clear the pipeline if used in a previous tutorial
pipeline = get_pipeline(pipeline_name)
pipeline.undeploy()
pipeline.clear()
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

 ok
Waiting for deployment - this will take up to 45s ................ ok


{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.124.8.26',
   'name': 'engine-5b4f59cfd9-jqzt4',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'passthrough-vc-fixed-shape-tensor-test',
      'status': 'Running',
      'version': '0a9b2c4e-b2c9-4d7a-b4fa-55283db149b8'}]},
   'model_statuses': {'models': [{'name': 'passthrough-vc-fixed-shape-tensor',
      'sha': 'b6392a54195f35db1d9b82b5adc7534c39d9d7c4f35cc855cb73e9ad4433ca23',
      'status': 'Running',
      'version': '80fe0702-f757-4df5-9566-4e1ee163c0aa'}]}}],
 'engine_lbs': [{'ip': '10.124.0.140',
   'name': 'engine-lb-6b59985857-rm9zk',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': [{'ip': '10.124.0.141',
   'name': 'engine-sidekick-passthrough-vc-fixed-shape-tensor-306-589d7fwks',
   'status': 'Running',
   'reason': None,
   'details': [],
   'statuses': '\n'}]}

### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [17]:
image = np.zeros((1, 2052, 2456)).astype(np.uint16)

In [18]:
image.shape

(1, 2052, 2456)

In [19]:
input_data = pa.Table.from_pydict({
    'image': pa.FixedShapeTensorArray.from_numpy_ndarray(image),
})
input_data

pyarrow.Table
image: extension<arrow.fixed_shape_tensor>
----
image: [[[0,0,0,0,0,...,0,0,0,0,0]]]

In [20]:
pipeline.infer(input_data, timeout=600)

pyarrow.Table
time: timestamp[ms]
in.image.data: list<item: uint16>
  child 0, item: uint16
in.image.shape: fixed_size_list<item: int32>[2]
  child 0, item: int32
out.image.data: list<item: uint16>
  child 0, item: uint16
out.image.shape: fixed_size_list<item: int32>[2]
  child 0, item: int32
out.mask.data: list<item: uint16>
  child 0, item: uint16
out.mask.shape: fixed_size_list<item: int32>[2]
  child 0, item: int32
out.mask_overlay.data: list<item: uint8>
  child 0, item: uint8
out.mask_overlay.shape: fixed_size_list<item: int32>[3]
  child 0, item: int32
out.virtual_stain.data: list<item: uint8>
  child 0, item: uint8
out.virtual_stain.shape: fixed_size_list<item: int32>[3]
  child 0, item: int32
anomaly.count: uint32 not null
----
time: [[2024-07-02 15:21:59.205]]
in.image.data: [[[0,0,0,0,0,...,0,0,0,0,0]]]
in.image.shape: [[[2052,2456]]]
out.image.data: [[[0,0,0,0,0,...,0,0,0,0,0]]]
out.image.shape: [[[2052,2456]]]
out.mask.data: [[[0,0,0,0,0,...,0,0,0,0,0]]]
out.mask.shape: [[

### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [ ]:
pipeline.undeploy()